# ** Mouse mutability model --> only 760 muts tho...** 

** getting mouse genome ** 

In [1]:
!wget ftp://ftp.ensembl.org/pub/release-102/fasta/mus_musculus/dna/README

--2021-01-19 16:59:41--  ftp://ftp.ensembl.org/pub/release-102/fasta/mus_musculus/dna/README
           => “README”
Resolving ftp.ensembl.org... 193.62.197.76
Connecting to ftp.ensembl.org|193.62.197.76|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/release-102/fasta/mus_musculus/dna ... done.
==> SIZE README ... 5081
==> PASV ... done.    ==> RETR README ... done.
Length: 5081 (5.0K) (unauthoritative)

100%[======================================>] 5,081       --.-K/s   in 0.005s  

2021-01-19 16:59:42 (982 KB/s) - “README” saved [5081]



In [17]:
#yes we are using GRCm38 )what the paper said) 
!wget ftp://ftp.ensembl.org/pub/release-102/fasta/mus_musculus/dna/Mus_musculus.GRCm38.dna.primary_assembly.fa.gz

--2021-01-24 10:51:36--  ftp://ftp.ensembl.org/pub/release-102/fasta/mus_musculus/dna/Mus_musculus.GRCm38.dna.primary_assembly.fa.gz
           => “Mus_musculus.GRCm38.dna.primary_assembly.fa.gz”
Resolving ftp.ensembl.org... 193.62.193.139
Connecting to ftp.ensembl.org|193.62.193.139|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/release-102/fasta/mus_musculus/dna ... done.
==> SIZE Mus_musculus.GRCm38.dna.primary_assembly.fa.gz ... 805984352
==> PASV ... done.    ==> RETR Mus_musculus.GRCm38.dna.primary_assembly.fa.gz ... done.
Length: 805984352 (769M) (unauthoritative)

100%[======================================>] 805,984,352 31.0M/s   in 38s     

2021-01-24 10:52:15 (20.4 MB/s) - “Mus_musculus.GRCm38.dna.primary_assembly.fa.gz” saved [805984352]



** import the mutation data ** 

In [27]:
import pandas as pd 
import json 

In [28]:
mouse_muts = pd.read_csv("41467_2019_12023_MOESM5_ESM.csv")

In [41]:
mouse_muts.columns = ['chr', 'pos', 'type', 'ref', 'alt', 'called in',
       'n_individuals shared', 'shared', 'Mosaic', 'VEE/Zygotic', 'Lineage',
       'RP_Phasing', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14',
       'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18',
       'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22',
       'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26',
       'Unnamed: 27'] # comes with 

** creating the empty dictionary to fill ** 

In [84]:
bases = ["A", "T", "C", "G"]
mouse_mut_dict = {}
for b1 in bases: 
    for b2 in bases: 
        for b3 in bases: 
            triplet = b1+b2+b3
            mouse_mut_dict[triplet] = [0,{}]
            
            into_trips = []
            for b2_into in bases: 
                if b2_into != b2: 
                    into_triplet = b1+b2_into+b3
                    mouse_mut_dict[triplet][1][into_triplet] = 0 # now the dict has [XAX] = [0,{XTX:0, XGX:0, XCX:0}] format 

** working on the loop **

In [85]:
with gzip.open("mus_musculus_genome/Mus_musculus.GRCm38.dna.primary_assembly.fa.gz", "rt") as handle: #adding to the mouse_mut_df by chrom
    for record in SeqIO.parse(handle, "fasta"):
        if record.id not in [str(i) for i in range(1, 20)]:         #skipping MT/X other dna types (only want autosomes) (we only have autosomes in mut dataset)
            break 
        chrom_mut_df = mouse_muts[mouse_muts.chr == int(record.id)] #filter the muts df by the chrtom we are currently on 
        chrom_dna = str(record.seq)
        for index, row in chrom_mut_df.iterrows():
            mut_pos = row.pos-1                                     # note that the mut df is 1-based so need -1 to convert to python 0-base 
            ref_triplet = str(chrom_dna[mut_pos-1:mut_pos+2])       #the reference triplet fgrom the fasta file 
            alt_triplet = ref_triplet[0]+row.alt+ref_triplet[2]     #alt triplet using the alt base in data set 
            if row.ref != chrom_dna[mut_pos]:                       #emergency flag: making sure our dataset matches the fasta
                print("shite")
                break
            mouse_mut_dict[ref_triplet][0]+=1                       #adding the "chosen" base in the mouse_mut_dict 
            mouse_mut_dict[ref_triplet][1][alt_triplet]+=1          #adding the "into" base in the sub dict 


In [87]:
tmp_mut_sum = 0
for value in mouse_mut_dict.values(): 
    tmp_mut_sum+= value[0]

In [88]:
tmp_mut_sum

760

In [91]:
with open('muts_2021_01_24_Mus_musculus.txt', 'w') as outfile:
    json.dump(mouse_mut_dict, outfile)